In [ ]:
from flask import Flask, render_template
from flask import *
import os
from werkzeug.utils import secure_filename
from  tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

app = Flask(__name__)

# Classes of trafic signs
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing vehicle over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Vehicle > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing vehicle > 3.5 tons' }

def image_processing(img):
    model = load_model('./model/traffic_classifier_main64_model.h5')
    data=[]
    image = Image.open(img)
    #convert to jpg
    image = image.convert('RGB')
    image = image.resize((64,64))
    data.append(np.array(image))
    X_test=np.array(data)
    #Y_pred = model.predict_classes(X_test)
    Y_pred = np.argmax(model.predict(X_test), axis=-1)
    return Y_pred

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/traffic')
def index():
    return render_template('index.html')

@app.route('/dataset')
def data():
    return render_template('dataset.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']
        file_path = secure_filename(f.filename)
        f.save(file_path)

        # Make prediction
        result = image_processing(file_path)
        s = [str(i) for i in result]
        a = int("".join(s))
        result = "Predicted Traffic Sign is: " +classes[a]
        os.remove(file_path)
        return result
    return None

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2021 08:23:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:00] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:00] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:01] "GET /static/css/img/traffic-light.jpg HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:01] "GET /static/css/img/icon_t.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:10] "GET /traffic HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:24:10] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Dec/2021 08:24:10] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Dec/2021 08:24:10] "GET /static/css/img/traffic-light.jpg HTTP/1.1" 304 -
127.0.0.1 - - [23/Dec/2021 08:24:17] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Dec/2021 08:25:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 08:25:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23